In [5]:
import pandas as pd
import dgl
import dgl.nn
import os
import numpy as np
import torch

In [6]:
data_dir = '/home/ZYH/ISM/lasftm_asia/'
edges_path = os.path.join(data_dir, 'lastfm_asia_edges.csv')
nodes_feature_path = os.path.join(data_dir, 'lastfm_asia_features.txt')
nodes_label_path = os.path.join(data_dir, 'lastfm_asia_target.csv')
f = open(nodes_feature_path, "r")
node_feature_dict = dict(eval(f.read()))
edges = pd.read_csv(edges_path)
sources_node = np.array(edges['node_1'])
dist_nodes = np.array(edges['node_2'])
nodes_label = pd.read_csv(nodes_label_path)
# create graph
num_nodes = len(nodes_label)
g = dgl.DGLGraph((sources_node, dist_nodes), num_nodes=num_nodes)
feature_len_list = []
for key,value in node_feature_dict.items():
    feature_len_list.append(len(value))
feature_list = np.zeros((num_nodes, max(feature_len_list)))
for i, (key,value) in enumerate(node_feature_dict.items()):
    feature_list[i][:len(value)] = np.array(value)
g.ndata['feat'] = torch.tensor(feature_list).float()
g.ndata['label'] = torch.tensor(np.array(nodes_label['target']))
g.ndata['train_mask'] = torch.cat((torch.tensor([True]*5000), torch.tensor([False]*2624)),dim=-1)
g.ndata['val_mask'] = torch.cat((torch.tensor([False]*5000), torch.tensor([True]*1000),torch.tensor([False]*1624)),dim=-1)
g.ndata['test_mask'] = torch.cat((torch.tensor([False]*6000), torch.tensor([True]*1624)),dim=-1)
g = dgl.add_self_loop(g)


In [7]:
g


Graph(num_nodes=7624, num_edges=35430,
      ndata_schemes={'feat': Scheme(shape=(944,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [33]:
from dgl.nn import GraphConv
import torch.nn as nn

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
        

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 1024, 18)

In [34]:
import torch.nn.functional as F
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    loss_fn = nn.CrossEntropyLoss()
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        print(labels[train_mask])

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = loss_fn(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, 18)
train(g, model)

tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 0, loss: 1295.016, val acc: 0.019 (best 0.019), test acc: 0.019 (best 0.019)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 5, loss: 3.263, val acc: 0.211 (best 0.211), test acc: 0.197 (best 0.197)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 10, loss: 2.956, val acc: 0.214 (best 0.214), test acc: 0.199 (best 0.199)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 15, loss: 2.879, val acc: 0.214 (best 0.214), test acc: 0.199 (best 0.199)
tensor([ 8, 17,  3,  ...,  6, 10,  6])


In [8]:
from dgl.nn import TAGConv
import torch.nn as nn

class TAG(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(TAG, self).__init__()
        self.conv1 = TAGConv(in_feats, h_feats)
        self.conv2 = TAGConv(h_feats, num_classes)
        

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = TAG(g.ndata['feat'].shape[1], 1024, 18)

In [11]:
import torch.nn.functional as F
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    loss_fn = nn.CrossEntropyLoss()
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        print(labels[train_mask])

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = loss_fn(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = TAG(g.ndata['feat'].shape[1], 16, 18)
train(g, model)

tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 0, loss: 8697.974, val acc: 0.050 (best 0.050), test acc: 0.043 (best 0.043)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 5, loss: 8076.185, val acc: 0.154 (best 0.154), test acc: 0.158 (best 0.158)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 10, loss: 2.965, val acc: 0.213 (best 0.213), test acc: 0.198 (best 0.199)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 15, loss: 2.848, val acc: 0.215 (best 0.215), test acc: 0.199 (best 0.199)
tensor([ 8, 17,  3,  ...,  6, 10,  6

In [14]:
from dgl.nn import EdgeConv
import torch.nn as nn

class EDGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(EDGE, self).__init__()
        self.conv1 = EdgeConv(in_feats, h_feats)
        self.conv2 = EdgeConv(h_feats, num_classes)
        

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = EDGE(g.ndata['feat'].shape[1], 1024, 18)

In [15]:
import torch.nn.functional as F
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    loss_fn = nn.CrossEntropyLoss()
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        print(labels[train_mask])

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = loss_fn(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = EDGE(g.ndata['feat'].shape[1], 16, 18)
train(g, model)

tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 0, loss: 1745.219, val acc: 0.054 (best 0.054), test acc: 0.050 (best 0.050)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 5, loss: 321.238, val acc: 0.098 (best 0.123), test acc: 0.087 (best 0.109)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 10, loss: 29.030, val acc: 0.088 (best 0.123), test acc: 0.085 (best 0.109)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 15, loss: 4.991, val acc: 0.166 (best 0.166), test acc: 0.161 (best 0.161)
tensor([ 8, 17,  3,  ...,  6, 10,  6

In [17]:
from dgl.nn import SAGEConv
import torch.nn as nn

class SAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(SAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats,'pool')
        self.conv2 = SAGEConv(h_feats, num_classes,'pool')
        

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = SAGE(g.ndata['feat'].shape[1], 1024, 18)

In [18]:
import torch.nn.functional as F
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    loss_fn = nn.CrossEntropyLoss()
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        print(labels[train_mask])

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = loss_fn(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = SAGE(g.ndata['feat'].shape[1], 16, 18)
train(g, model)

tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 0, loss: 30738.602, val acc: 0.024 (best 0.024), test acc: 0.033 (best 0.033)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 5, loss: 4.500, val acc: 0.131 (best 0.131), test acc: 0.134 (best 0.134)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 10, loss: 2.838, val acc: 0.132 (best 0.132), test acc: 0.137 (best 0.137)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 15, loss: 2.799, val acc: 0.132 (best 0.132), test acc: 0.137 (best 0.137)
tensor([ 8, 17,  3,  ...,  6, 10,  6])

In [29]:
from dgl.nn import AGNNConv
import torch.nn as nn

class AGNN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(AGNN, self).__init__()
        self.conv1 = AGNNConv()
        self.conv2 = AGNNConv()

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = AGNN(g,g.ndata['feat'].shape[1],18)

In [30]:
import torch.nn.functional as F
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    loss_fn = nn.CrossEntropyLoss()
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        print(labels[train_mask])

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = loss_fn(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = AGNN(g,g.ndata['feat'].shape[1], 18)
train(g, model)

tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 0, loss: 2912.349, val acc: 0.028 (best 0.028), test acc: 0.030 (best 0.030)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 5, loss: 2911.081, val acc: 0.028 (best 0.028), test acc: 0.030 (best 0.030)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 10, loss: 2909.865, val acc: 0.028 (best 0.028), test acc: 0.030 (best 0.030)
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
tensor([ 8, 17,  3,  ...,  6, 10,  6])
In epoch 15, loss: 2908.721, val acc: 0.027 (best 0.028), test acc: 0.031 (best 0.030)
tensor([ 8, 17,  3,  ...,  6, 